In [ ]:
from dotenv import load_dotenv
import os
from langsmith import traceable

# Cargar el archivo .env
load_dotenv('../.env')

In [ ]:
import os
from typing import Dict, Any, List, Optional
from pydantic import BaseModel
from typing_extensions import TypedDict, Annotated

from langchain_core.language_models import BaseChatModel
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, BaseMessage
from typing import Any as _Any
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_openai import OpenAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from langchain_core.runnables import RunnablePassthrough
import logging

# Qdrant

In [ ]:
import os
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore

qdrant_url = os.getenv('QDRANT_URL')
qdrant_key = os.getenv('QDRANT_KEY')
collection_name = os.getenv('QDRANT_COLLECTION_NAME')

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

print(f"[Qdrant] URL: {qdrant_url} | Collection: {collection_name}")
client = QdrantClient(
    url=qdrant_url,
    api_key=qdrant_key
)

qdrant_store = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=embeddings
)

retriever = qdrant_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 10}
)

message_history_limit = 10

# Conexion con BD

In [ ]:
import os
from typing import Optional
from psycopg_pool import ConnectionPool

def init_pool(min_size: int = 1, max_size: int = 10) -> ConnectionPool:
    host = os.getenv("POSTGRES_HOST")
    port = os.getenv("POSTGRES_PORT", "5432")
    user = os.getenv("POSTGRES_USER")
    password = os.getenv("POSTGRES_PASSWORD")
    database = os.getenv("POSTGRES_TECHFLOW_DATABASE")
    sslmode = os.getenv("POSTGRES_SSLMODE", "require")
    
    # Construir cadena de conexión (usar dbname en lugar de database)
    dsn = f"host={host} port={port} user={user} password={password} dbname={database} sslmode={sslmode}"
    
    _pool = ConnectionPool(dsn, min_size=min_size, max_size=max_size)
    return _pool

# Grafo

## Modelo

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

## Tools

In [ ]:
import os
from google.oauth2.service_account import Credentials
from google.auth import default
from googleapiclient.discovery import build
from datetime import datetime
from typing import Optional

GOOGLE_SHEETS_SPREADSHEET_ID = os.getenv("GOOGLE_SHEETS_SPREADSHEET_ID")
GOOGLE_SHEETS_NAME = os.getenv("GOOGLE_SHEETS_NAME")
GOOGLE_CREDENTIALS_FILE = os.getenv("GOOGLE_CREDENTIALS_FILE")

def registrar_cliente(email: str, nombres: str, apellidos: str, numero_documento: str, telefono: str):
    """
    Registra un nuevo cliente

    Args:
        email: Correo electrónico del usuario
        nombres: Nombre del usuario
        apellidos: Apellido del usuario
        numero_documento: Número de documento
        telefono: Número de teléfono

    Returns:
        str: 'ok' si la operación fue exitosa
    """
    try:
        # Configuración de credenciales
        credentials_file = GOOGLE_CREDENTIALS_FILE
        spreadsheet_id = GOOGLE_SHEETS_SPREADSHEET_ID
        sheet_name = GOOGLE_SHEETS_NAME

        # Scopes necesarios
        scopes = [
            'https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/drive.file'
        ]

        # Autenticación
        if credentials_file and os.path.exists(credentials_file):
            # Local: usar archivo JSON
            credentials = Credentials.from_service_account_file(credentials_file, scopes=scopes)
            print("Usando archivo JSON local")
        else:
            # Cloud Run: usar Application Default Credentials
            credentials, _ = default(scopes=scopes)
            
        print("Usando Application Default Credentials")
        #credentials = Credentials.from_service_account_file(scopes=scopes)
        service = build('sheets', 'v4', credentials=credentials)

        fecha_registro = datetime.now().strftime('%d/%m/%Y %H:%M:%S')

        # Preparar los datos de la nueva fila
        new_row = [
            fecha_registro,
            email,
            nombres,
            apellidos,
            numero_documento,
            telefono
        ]

        # Rango donde agregar la nueva fila
        range_to_append = f"{sheet_name}!A:F"

        # Cuerpo de la petición
        body = {
            'values': [new_row]
        }

        # Encontrar la próxima fila vacía para evitar copiar formato
        # Primero obtenemos los datos existentes para saber cuántas filas hay
        existing_data = service.spreadsheets().values().get(
            spreadsheetId=spreadsheet_id,
            range=f"{sheet_name}!A:A"
        ).execute()

        # Calcular la próxima fila vacía
        existing_rows = len(existing_data.get('values', []))
        next_row = existing_rows + 1

        # Insertar en la fila específica (esto evita copiar formato)
        specific_range = f"{sheet_name}!A{next_row}:F{next_row}"

        result = service.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id,
            range=specific_range,
            valueInputOption='RAW',
            body=body
        ).execute()

        rows_added = result.get("updates", {}).get("updatedRows", 0)
        print(f'Usuario registrado correctamente')
        print(f'Fecha de registro: {fecha_registro}')

        return 'ok'

    except Exception as error:
        print(f'Error al registrar el usuario: {error}')
        raise error

def contar_registros():
    """
    Cuenta cuántos registros (filas) hay en el Google Sheet

    Returns:
        str: Mensaje con el número de registros
    """
    include_headers = False

    # Configuración de credenciales
    credentials_file = GOOGLE_CREDENTIALS_FILE
    spreadsheet_id = GOOGLE_SHEETS_SPREADSHEET_ID
    sheet_name = GOOGLE_SHEETS_NAME
    
    try:
        # Scopes necesarios para Google Sheets
        scopes = [
            'https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/drive.file'
        ]

        # Autenticación
        if credentials_file and os.path.exists(credentials_file):
            # Local: usar archivo JSON
            credentials = Credentials.from_service_account_file(credentials_file, scopes=scopes)
            print("Usando archivo JSON local")
        else:
            # Cloud Run: usar Application Default Credentials
            credentials, _ = default(scopes=scopes)

        # Crear el servicio de Google Sheets
        service = build('sheets', 'v4', credentials=credentials)

        # Obtener todas las filas con datos (columna A como referencia)
        result = service.spreadsheets().values().get(
            spreadsheetId=GOOGLE_SHEETS_SPREADSHEET_ID,
            range=f"{GOOGLE_SHEETS_NAME}!A:A"
        ).execute()

        # Contar filas con datos
        rows_with_data = result.get('values', [])
        total_rows = len(rows_with_data)

        if include_headers:
            registros = total_rows
            print(f'Total de filas (incluyendo headers): {registros}')
        else:
            # Restar 1 para excluir la fila de headers
            registros = max(0, total_rows - 1)
            print(f'Total de registros (sin headers): {registros}')
            print(f'Total de filas (con headers): {total_rows}')

        return f"Hay {registros} clientes registrados"

    except Exception as error:
        print(f'Error al contar registros: {error}')
        raise error

def get_current_date() -> dict:
    """
    Obtener la fecha actual en el formato YYYY-MM-DD
    """
    return {"current_date": datetime.now().strftime("%Y-%m-%d")}

def get_program_price(program_name: str) -> dict:
    """Obtiene el precio de un programa mediante su nombre.

    Parameters
    ----------
    program_name: str
        Nombre del programa a buscar.

    Returns
    -------
    dict
        {"program_price": float | None}
    """
    pool = init_pool()
    print(f"Buscando precio del programa: {program_name}")

    query = (
        """
        SELECT precio
        FROM public.programas
        WHERE nombre ILIKE '%%' || %s || '%%'
        LIMIT 1
        """
    )

    try:
        with pool.connection() as connection:
            with connection.cursor() as cursor:
                cursor.execute(query, (program_name,))
                row: Optional[tuple] = cursor.fetchone()
                if not row:
                    return {"program_price": None}
                price_value = row[0]
                try:
                    # Asegurar retorno numérico float
                    price_float = float(price_value) if price_value is not None else None
                except Exception:
                    price_float = None
                print(f"Precio del programa: {price_float}")
                return {"program_price": price_float}
    

    except Exception as exc:
        # Log simple; en entornos reales usar logger
        print(f"[get_program_price] Error consultando precio: {exc}")
        return {"program_price": None}
    

### Prueba de tools

In [ ]:
print(get_program_price("Data architect"))

## Model con tools

In [ ]:
model = llm.bind_tools([
    registrar_cliente,
    contar_registros,
    get_current_date,
    get_program_price,
])

In [ ]:
# Prueba
resp_tool = model.invoke("cual es el costo del curso Arquitectura de Datos Empresarial?")
print(resp_tool.tool_calls)

## Estado

In [ ]:
class AdaptiveAgentState(TypedDict):
    """
    Estado personalizado que incluye mensajes y metadatos del RAG adaptativo
    """
    messages: Annotated[List[BaseMessage], add_messages]
    complexity_level: Optional[str]
    retrieved_docs: Optional[List[Any]]
    question: Optional[str]  # Pregunta actual (original o reformulada)
    generation: Optional[str]  # Respuesta generada
    answer_evaluation: Optional[str]  # Resultado de evaluación de la respuesta ('yes'/'no')
    max_retries: Optional[int]  # Máximo número de reintentos
    retry_count: Optional[int]  # Contador de reintentos
    rag_processs: Optional[str]  # Proceso de RAG ('rag'/'no_rag')

## Prompts

In [ ]:
COMPLEXITY_PROMPT = """Eres un experto clasificando la complejidad de preguntas de estudiantes de TechFlow Academy.

Clasifica la pregunta en uno de estos 3 niveles:

SIMPLE: Saludos, consultas básicas sobre horarios, ubicación, contacto, despedidas
- Ejemplos: "Hola", "¿Están abiertos?", "¿Dónde quedan?", "Gracias", "Adiós"

RAG: Preguntas sobre programas, cursos, profesores, metodología, contenido académico
- Ejemplos: "¿Qué incluye Data Engineering?", "¿Quién enseña ML?", "¿Cómo son las clases?"

TOOLS: Preguntas sobre costos, inscripciones, disponibilidad de cupos, registro
- Ejemplos: "¿Cuánto cuesta?", "¿Hay cupos en Data Science?", "Quiero inscribirme"

Retorna JSON con clave 'complexity_level' y valor 'simple', 'rag' o 'tools'.

Pregunta: {question}"""

TOOL_ROUTER_PROMPT = """Eres un router que determina qué herramienta usar según la pregunta del estudiante.

Herramientas disponibles:
- get_course_cost: Para preguntas sobre precios, costos, valores de programas
- get_student_count: Para preguntas sobre cuántos estudiantes, disponibilidad, cupos
- register_student: Para inscripciones, registros, matriculas

Ejemplos:
"¿Cuánto cuesta Data Engineering?" → get_course_cost
"¿Hay cupos disponibles?" → get_student_count  
"Quiero inscribirme en ML Engineer" → register_student

Retorna JSON con clave 'tool_name' y el nombre de la herramienta.

Pregunta: {question}"""

SIMPLE_PROMPT = """Eres un asistente amigable de TechFlow Academy, un instituto de programación y ciencia de datos en Lima, Perú.

Para consultas simples como saludos, ubicación, horarios básicos, contacto:
- Responde de forma cordial y directa
- Menciona que TechFlow Academy es especialista en Data Engineering, ML Engineering, Data Visualization, etc.
- Para consultas específicas sobre programas, costos o inscripciones, indica que puedes ayudar con información detallada
- Mantén un tono profesional pero cercano

Información básica:
- Horarios: Lunes a viernes 7AM-10PM, Sábados 8AM-8PM
- Sedes: Miraflores, San Isidro, La Molina, Surco
- Modalidades: Virtual, Presencial, Híbrida
- WhatsApp: Canal principal de comunicación"""

RAG_PROMPT = """Eres un asistente especializado de TechFlow Academy. 

Responde la pregunta basándote únicamente en el contexto proporcionado sobre nuestros programas, profesores, metodología y servicios.

Instrucciones:
- Usa solo la información del contexto
- Si no encuentras información específica, indica que puedes ayudar de otra manera
- Mantén respuestas claras y estructuradas
- Incluye detalles relevantes como duración, modalidades, requisitos
- Sugiere próximos pasos cuando sea apropiado (ej: "¿Te gustaría conocer los costos?")"""

TOOL_PROMPT = """Eres un asistente de TechFlow Academy especializado en información sobre costos, inscripciones y disponibilidad.

Genera una respuesta natural y útil basada en el resultado de la herramienta consultada.

Instrucciones:
- Presenta la información de forma clara y estructurada
- Incluye próximos pasos o acciones recomendadas
- Mantén tono profesional pero amigable
- Si es información de costos, menciona opciones de financiamiento
- Si es sobre disponibilidad, sugiere alternativas si es necesario
- Si es registro, confirma y explica siguientes pasos"""

MODEL_SYSTEM_MESSAGE = """Eres un asistente especializado de TechFlow Academy, un instituto de programación y ciencia de datos en Lima, Perú.

Tu función es ayudar a estudiantes potenciales y actuales con información sobre programas, inscripciones y servicios.

Instrucciones:
- Mantén respuestas claras y profesionales
- Usa las herramientas disponibles cuando sea necesario para obtener información actualizada
- Si necesitas registrar un estudiante, asegúrate de obtener todos los datos requeridos
- Para consultas sobre costos o cupos, usa las herramientas correspondientes
- Proporciona información útil y sugiere próximos pasos cuando sea apropiado

Herramientas disponibles:
- registrar_cliente: Para registrar información de estudiantes interesados
- contar_registros: Para consultar cuántos estudiantes están registrados
- get_current_date: Para obtener la fecha actual

Información básica de TechFlow Academy:
- Horarios: Lunes a viernes 7AM-10PM, Sábados 8AM-8PM
- Sedes: Miraflores, San Isidro, La Molina, Surco
- Modalidades: Virtual, Presencial, Híbrida
- Programas: Data Engineering, ML Engineering, Data Visualization, Data Science"""

# Prompts para RAG Adaptativo

GRADE_DOCUMENTS_PROMPT = """Eres un evaluador que determina si los documentos recuperados son relevantes para responder la pregunta del estudiante.

Evalúa cada documento y determina si contiene información útil para responder la pregunta.

Documentos relevantes son aquellos que:
- Contienen información directamente relacionada con la pregunta
- Proporcionan contexto útil para formular una respuesta completa
- Incluyen detalles específicos sobre programas, profesores, metodología, etc.

Documentos NO relevantes son aquellos que:
- No tienen relación con la pregunta
- Contienen información genérica sin valor específico
- No aportan contexto útil para la respuesta

Pregunta: {question}

Documentos a evaluar: {documents}

Responde con 'yes' si al menos uno de los documentos es relevante, o 'no' si ninguno es relevante."""

EVALUATE_ANSWER_PROMPT = """Eres un evaluador que determina si una respuesta generada responde adecuadamente la pregunta del estudiante.

Evalúa si la respuesta:
- Responde directamente la pregunta formulada
- Proporciona información específica y útil
- Está basada en el contexto proporcionado
- No contiene información inventada o alucinada
- Es clara y comprensible

Pregunta: {question}

Respuesta generada: {generation}

Documentos de contexto: {documents}

Responde con 'yes' si la respuesta es adecuada, o 'no' si necesita mejoras."""

REWRITE_QUESTION_PROMPT = """Eres un experto en reformular preguntas para mejorar la recuperación de documentos relevantes.

La pregunta original no obtuvo documentos relevantes. Reescribe la pregunta para:
- Usar términos más específicos relacionados con programación y ciencia de datos
- Incluir sinónimos o términos alternativos
- Hacer la pregunta más clara y específica
- Mantener la intención original pero con mejor búsqueda

Pregunta original: {question}

Genera una pregunta reformulada que mejore la recuperación de documentos relevantes."""

WEB_SEARCH_PROMPT = """La información en nuestra base de conocimientos no fue suficiente para responder esta pregunta sobre TechFlow Academy.

Genera una respuesta útil reconociendo las limitaciones y sugiriendo próximos pasos:

- Indica que la información específica no está disponible en este momento
- Sugiere contactar directamente para información más detallada
- Proporciona información general que sí conoces sobre TechFlow Academy
- Mantén un tono profesional y servicial

Pregunta: {question}

Contexto disponible: {documents}"""

## Nodos

In [ ]:
class DocumentRelevance(BaseModel):
    """Modelo para evaluación de relevancia de documentos"""
    decision: str  # 'yes' o 'no'

class ComplexityLevel(BaseModel):
    """Modelo para clasificación de complejidad"""
    complexity_level: str

class AnswerEvaluation(BaseModel):
    """Modelo para evaluación de calidad de respuesta"""
    decision: str  # 'yes' o 'no'

class QuestionRewrite(BaseModel):
    """Modelo para re-escritura de preguntas"""
    rewritten_question: str

In [ ]:
def get_last_messages(state: "AdaptiveAgentState", limit: Optional[int] = None) -> List[BaseMessage]:
    effective_limit = limit if isinstance(limit, int) and limit > 0 else message_history_limit
    messages = state.get("messages", [])
    if not messages:
        return []
    if len(messages) <= effective_limit:
        return messages
    return messages[-effective_limit:]

def format_docs(docs):
        """Format documents for RAG context"""
        return "\n\n".join(doc.page_content for doc in docs)

#@traceable(name="classify_complexity")
def classify_complexity(state: AdaptiveAgentState):
    """Classify question complexity based on last human message"""
    print("---CLASSIFY COMPLEXITY---")
    
    complexity_prompt = PromptTemplate(template=COMPLEXITY_PROMPT, input_variables=["question"])
    complexity_classifier = (
        complexity_prompt 
        | llm.with_structured_output(ComplexityLevel)
    )
    
    # Extraer la última pregunta del usuario de los mensajes
    last_human_message = None
    for msg in reversed(state["messages"]):
        if isinstance(msg, HumanMessage):
            last_human_message = msg.content
            break
    
    if not last_human_message:
        # Si no hay mensaje humano, usar mensaje por defecto
        complexity_level = "simple"
    else:
        result = complexity_classifier.invoke({"question": last_human_message})
        complexity_level = result.complexity_level
    
    print(f"Complexity Level: {complexity_level}")
    # Retornar el estado actualizado con el nivel de complejidad
    return {
        "complexity_level": complexity_level,
        "retrieved_docs": state.get("retrieved_docs", None)
    }

#@traceable(name="simple_response") 
def simple_response(state: AdaptiveAgentState):
    """Generate simple response using full chat history"""
    print("---SIMPLE RESPONSE---")

    # Prepend a system message and pass the full history to the chat model
    messages_for_model: List[BaseMessage] = [
        SystemMessage(content=SIMPLE_PROMPT)
    ] + get_last_messages(state)

    response = llm.invoke(messages_for_model)
    return {"messages": [response]}

#@traceable(name="rag_retrieve")
def rag_retrieve(state: AdaptiveAgentState):
    """Retrieve documents for RAG"""
    print("---RAG RETRIEVE---")
    
    # Obtener la pregunta (original o reformulada)
    question = state.get("question")
    if not question:
        # Extraer la última pregunta del usuario
        for msg in reversed(state["messages"]):
            if isinstance(msg, HumanMessage):
                question = msg.content
                break
    
    if not question:
        question = "información general"
    
    # Recuperar documentos
    print(f"Recuperando contexto de pregunta: {question}")
    documents = retriever.get_relevant_documents(question)

    print(f"[RAG] Docs recuperados: {len(documents)}")
    for i, doc in enumerate(documents):
        preview = doc.page_content.replace('\n',' ')[:220]
        print(f"[{i}] {preview}")
        print(f"    meta: {doc.metadata}")

    # Retornar estado actualizado con documentos y pregunta
    return {
        "retrieved_docs": documents,
        "question": question,
        "max_retries": state.get("max_retries", 3),
        "retry_count": state.get("retry_count", 0)
    }

def grade_documents(state: AdaptiveAgentState):
    grade_docs_prompt = PromptTemplate(template=GRADE_DOCUMENTS_PROMPT, input_variables=["question", "documents"])
    document_grader = (
        grade_docs_prompt 
        | llm.with_structured_output(DocumentRelevance)
    )

    """Grade document relevance to question"""
    print("---GRADE DOCUMENTS---")
    
    question = state.get("question", "")
    documents = state.get("retrieved_docs", [])
    
    if not documents:
        print("No documents to grade")
        return {"retrieved_docs": []}
    
    # Formatear documentos para evaluación
    docs_text = format_docs(documents)
    
    # Evaluar relevancia
    result = document_grader.invoke({"question": question, "documents": docs_text})
    print(f"Document relevance: {result.decision}")
    
    if result.decision == "yes":
        print("Documents are relevant")
        return {"retrieved_docs": documents}
    else:
        print("Documents are not relevant")
        return {"retrieved_docs": []}

def rag_generate(state: AdaptiveAgentState):
    """Generate RAG response using full chat history + context"""
    print("---RAG GENERATE---")

    # Obtener documentos recuperados
    documents = state.get("retrieved_docs", [])
    context_text = format_docs(documents) if documents else ""

    # Prepend system with instructions and context, then pass full history
    system_content = f"{RAG_PROMPT}\n\nContexto:\n{context_text}"
    messages_for_model: List[BaseMessage] = [
        SystemMessage(content=system_content)
    ] + get_last_messages(state)

    response = llm.invoke(messages_for_model)
    
    # Almacenar la respuesta generada para evaluación
    return {
        "messages": [response],
        "generation": response.content
    }

def evaluate_answer(state: AdaptiveAgentState):
    """Evaluate if the generated answer is adequate"""
    print("---EVALUATE ANSWER---")

    evaluate_answer_prompt = PromptTemplate(template=EVALUATE_ANSWER_PROMPT, input_variables=["question", "generation", "documents"])
    answer_evaluator = (
        evaluate_answer_prompt 
        | llm.with_structured_output(AnswerEvaluation)
    )
    
    question = state.get("question", "")
    generation = state.get("generation", "")
    documents = state.get("retrieved_docs", [])
    docs_text = format_docs(documents) if documents else ""
    
    # Evaluar la respuesta
    result = answer_evaluator.invoke({
        "question": question,
        "generation": generation,
        "documents": docs_text
    })
    
    print(f"Answer evaluation: {result.decision}")
    return {
        "generation": generation,
        "answer_evaluation": result.decision  # Almacenar el resultado de la evaluación
    }

def rewrite_question(state: AdaptiveAgentState):
    """Rewrite question to improve retrieval"""
    print("---REWRITE QUESTION---")

    rewrite_question_prompt = PromptTemplate(template=REWRITE_QUESTION_PROMPT, input_variables=["question"])
    question_rewriter = (
        rewrite_question_prompt 
        | llm.with_structured_output(QuestionRewrite)
    )
    
    original_question = state.get("question", "")
    retry_count = state.get("retry_count", 0)
    
    # Re-escribir la pregunta
    result = question_rewriter.invoke({"question": original_question})
    rewritten_question = result.rewritten_question
    
    print(f"Question rewritten: {original_question} -> {rewritten_question}")
    
    return {
        "question": rewritten_question,
        "retry_count": retry_count + 1,
        "retrieved_docs": []  # Limpiar documentos para nueva búsqueda
    }

def web_search_fallback(state: AdaptiveAgentState):
    """Fallback when RAG fails multiple times"""
    print("---WEB SEARCH FALLBACK---")
    
    question = state.get("question", "")
    documents = state.get("retrieved_docs", [])
    docs_text = format_docs(documents) if documents else ""
    
    # Generar respuesta de fallback
    system_content = f"{WEB_SEARCH_PROMPT}"
    prompt_template = PromptTemplate(
        template=system_content + "\n\nPregunta: {question}\n\nContexto disponible: {documents}",
        input_variables=["question", "documents"]
    )
    
    chain = prompt_template | llm
    response = chain.invoke({"question": question, "documents": docs_text})
    
    return {"messages": [response]}

def call_model_with_tools(state: AdaptiveAgentState):
    """Call model with tools for complex queries"""
    print("---CALL MODEL WITH TOOLS---")
    
    # Usar el modelo con los últimos 10 mensajes
    response = model.invoke(get_last_messages(state))
    
    # Devolver solo el delta de mensajes
    return {
        "messages": [response]
    }

def route_by_complexity(state: AdaptiveAgentState):
    """Route based on complexity classification"""
    complexity_level = state.get("complexity_level", "simple")
    print(f"---ROUTE BY COMPLEXITY: {complexity_level}---")
    return complexity_level

def decide_to_generate_or_rewrite(state: AdaptiveAgentState):
    """Decide whether to generate answer or rewrite question based on document relevance"""
    documents = state.get("retrieved_docs", [])
    if documents:
        print("Documents are relevant, proceeding to generate")
        return "generate"
    else:
        retry_count = state.get("retry_count", 0)
        max_retries = state.get("max_retries", 3)
        if retry_count < max_retries:
            print(f"Documents not relevant, rewriting question (attempt {retry_count + 1}/{max_retries})")
            return "rewrite"
        else:
            print(f"Max retries reached ({max_retries}), using fallback")
            return "fallback"

def decide_to_finish_or_rewrite(state: AdaptiveAgentState):
    """Decide whether to finish or rewrite based on answer evaluation"""
    answer_evaluation = state.get("answer_evaluation", "yes")
    retry_count = state.get("retry_count", 0)
    max_retries = state.get("max_retries", 3)
    
    print(f"Answer evaluation decision: {answer_evaluation}")
    
    # Si la respuesta es buena, terminar
    if answer_evaluation == "yes":
        print("Answer is adequate, finishing")
        return "finish"
    
    # Si la respuesta no es buena y aún hay reintentos disponibles
    if retry_count < max_retries:
        print(f"Answer needs improvement, rewriting question (attempt {retry_count + 1}/{max_retries})")
        return "rewrite"
    
    # Si se alcanzó el máximo de reintentos, terminar de todas formas
    print(f"Max retries reached ({max_retries}), finishing with current answer")
    return "finish"

## Creación del grafo

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# MemorySaver para almacenar el estado de la conversación en el hilo actual
within_thread_memory = MemorySaver()

# Define the graph
builder = StateGraph(AdaptiveAgentState)

# Add nodes
builder.add_node("classify_complexity", classify_complexity)
builder.add_node("simple_response", simple_response)

# RAG adaptativo nodes
builder.add_node("rag_retrieve", rag_retrieve)
builder.add_node("grade_documents", grade_documents)
builder.add_node("rag_generate", rag_generate)
builder.add_node("evaluate_answer", evaluate_answer)
builder.add_node("rewrite_question", rewrite_question)
builder.add_node("web_search_fallback", web_search_fallback)

# Tools nodes
builder.add_node("call_model_with_tools", call_model_with_tools)
builder.add_node(
    "tools",
    ToolNode([registrar_cliente, contar_registros, get_current_date, get_program_price])
)

# Add edges
builder.add_edge(START, "classify_complexity")

# Conditional routing from complexity classifier
builder.add_conditional_edges(
    "classify_complexity",
    route_by_complexity,
    {
        "simple": "simple_response",
        "rag": "rag_retrieve", 
        "tools": "call_model_with_tools",
    },
)

# Simple path
builder.add_edge("simple_response", END)

# RAG adaptativo path
builder.add_edge("rag_retrieve", "grade_documents")
builder.add_conditional_edges(
    "grade_documents",
    decide_to_generate_or_rewrite,
    {
        "generate": "rag_generate",
        "rewrite": "rewrite_question",
        "fallback": "web_search_fallback"
    }
)

# Re-write loop
builder.add_edge("rewrite_question", "rag_retrieve")

# Generate and evaluate
builder.add_edge("rag_generate", "evaluate_answer")
builder.add_conditional_edges(
    "evaluate_answer",
    decide_to_finish_or_rewrite,
    {
        "finish": END,
        #"finish": "classify_complexity",
        "rewrite": "rewrite_question"
    }
)

# Fallback path
builder.add_edge("web_search_fallback", END)

# Tools path
builder.add_conditional_edges(
    "call_model_with_tools",
    tools_condition,
    {"tools": "tools", END: END}
)
builder.add_edge("tools", "call_model_with_tools")

# Se compila el grafo con nombre estable para asegurar que se recupere el estado tras reinicios
graph_name = os.getenv("GRAPH_NAME", "chatbot_graph_v1")

graph = builder.compile(
    checkpointer=within_thread_memory,
    name=graph_name,
)
graph

# Pruebas

In [ ]:
#@traceable(name="process_message", metadata={"component": "adaptive_rag_chatbot"})
def process_message(message: str, user_id: str) -> Dict[str, Any]:
    config = {"configurable": {"thread_id": user_id, "user_id": user_id, "checkpoint_ns": graph_name}}
    input_messages = [HumanMessage(content=message)]

    # Inicializar el estado con todos los campos requeridos
    initial_state = {
        "messages": input_messages,
        "complexity_level": None,
        "retrieved_docs": None,
        "question": None,
        "generation": None,
        "answer_evaluation": None,
        "max_retries": 3,
        "retry_count": 0
    }

    result = graph.invoke(initial_state, config)
    ai_message = result["messages"][-1]

    return ai_message.content

## Prueba 1

In [ ]:
respuesta = process_message("que programas hay?", "ernesto")

In [ ]:
print(respuesta)

## Prueba 2

In [ ]:
respuesta = process_message("cual es el costo del programa Data Engineer?", "ernesto")

In [ ]:
print(respuesta)

## Prueba 3

In [ ]:
respuesta = process_message("Que programas hay y cual es el costo de cada uno?", "ernesto")

In [ ]:
print(respuesta)

# RAG como Tool

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_techflow_academy_info", # nombre de la tool
    "Busca y devuelve información sobre Programas, cursos, docentes, empresas aliadas, preguntas frecuentes, testimonio de graduados, matriculas y reglamentos sobre el instituto Techflow Academy", # descripcion de la tool
)

tools = [
    retriever_tool,
    registrar_cliente,
    contar_registros,
    get_current_date,
    get_program_price
]

In [ ]:
model = llm.bind_tools(tools)

```python
class AdaptiveAgentState(TypedDict):
    """
    Estado personalizado que incluye mensajes y metadatos del RAG adaptativo
    """
    messages: Annotated[List[BaseMessage], add_messages]
    complexity_level: Optional[str]
    retrieved_docs: Optional[List[Any]]
    question: Optional[str]  # Pregunta actual (original o reformulada)
    generation: Optional[str]  # Respuesta generada
    answer_evaluation: Optional[str]  # Resultado de evaluación de la respuesta ('yes'/'no')
    max_retries: Optional[int]  # Máximo número de reintentos
    retry_count: Optional[int]  # Contador de reintentos
```

In [ ]:
# Nuevo nodo inicial
def agent(state: AdaptiveAgentState):
    """
    Invoca al modelo del agente para generar una respuesta basada en el estado actual. Dada
    la pregunta, decidirá si recuperar usando la herramienta retriever o simplemente terminar.

    Args:
        state (messages): El estado actual

    Returns:
        dict: El estado actual con la respuesta del agente agregada a los mensajes
    """
    print("---CALL AGENT---")
    messages = state["messages"]
    messages.append(HumanMessage(content="Responde la pregunta del usuario basado en la cadena de mensajes y la peticion que te hizo"))
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

## Nuevo grafo

In [ ]:
from typing import Literal
from langchain_core.messages import AIMessage, ToolMessage

def validar_rag(state: "AdaptiveAgentState") -> Literal["generate", "agent"]:
    """
    Valida si se usó la tool del retriever (RAG) para decidir el siguiente nodo.
    
    Si se ejecutó la tool 'retrieve_techflow_academy_info', va al nodo 'generate'.
    Si se ejecutaron otras tools, regresa al nodo 'agent'.
    """
    print("---VALIDAR RAG---")
    
    messages = state.get("messages", [])
    
    # Buscar el último mensaje AI que tenga tool_calls
    for message in reversed(messages):
        if isinstance(message, AIMessage) and hasattr(message, 'tool_calls') and message.tool_calls:
            # Verificar si alguna de las tool_calls es la del retriever
            for tool_call in message.tool_calls:
                tool_name = tool_call.get("name", "")
                print(f"Tool ejecutada: {tool_name}")
                
                if tool_name == "retrieve_techflow_academy_info":
                    print("Se detectó tool del retriever RAG -> ir a 'generate'")
                    return "generate"
            
            # Si llegamos aquí, se ejecutaron otras tools pero no el retriever
            print("Se ejecutaron otras tools (no RAG) -> regresar a 'agent'")
            return "agent"
    
    # Si no se encontraron tool_calls, por defecto regresar al agent
    print("No se encontraron tool_calls -> regresar a 'agent'")
    return "agent"

In [ ]:
def generate(state: AdaptiveAgentState):
    """
    Genera una respuesta basada en el contexto RAG recuperado
    """
    print("---GENERATE---")
    
    messages = state.get("messages", [])
    
    # Buscar el último ToolMessage que contenga el contexto del RAG
    rag_context = ""
    for message in reversed(messages):
        if hasattr(message, 'content') and isinstance(message.content, str):
            # Los resultados del retriever vienen en ToolMessage
            if "Documento:" in message.content or any(keyword in message.content.lower() for keyword in ["techflow", "programa", "curso"]):
                rag_context = message.content
                print(f"Contexto RAG encontrado: {rag_context[:200]}...")
                break
    
    # Si no encontramos contexto específico, usar el último mensaje de tool
    if not rag_context:
        for message in reversed(messages):
            if hasattr(message, 'content') and len(str(message.content)) > 50:
                rag_context = str(message.content)
                print(f"Usando último contenido como contexto: {rag_context[:200]}...")
                break
    
    # Crear prompt para generar respuesta basada en el contexto RAG
    system_prompt = f"""Eres un asistente especializado de TechFlow Academy.

Basándote en la siguiente información recuperada, genera una respuesta clara y útil para el usuario.

Contexto RAG:
{rag_context}

Instrucciones:
- Responde de manera clara y estructurada
- Usa solo la información proporcionada en el contexto
- Si la información es limitada, indícalo y sugiere formas de obtener más detalles
- Mantén un tono profesional y amigable"""

    # Obtener la pregunta original del usuario
    user_question = ""
    for message in messages:
        if isinstance(message, HumanMessage):
            user_question = message.content
    
    # Crear el mensaje para el modelo
    messages_for_model = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_question)
    ]
    
    response = llm.invoke(messages_for_model)
    return {"messages": [response]}

from langgraph.checkpoint.memory import MemorySaver

# MemorySaver para almacenar el estado de la conversación en el hilo actual
within_thread_memory = MemorySaver()

# Define the graph
builder = StateGraph(AdaptiveAgentState)

# Add nodes
builder.add_node("agent", agent)
execute = ToolNode(tools)
builder.add_node("execute", execute)  # retrieval
builder.add_node("generate", generate)  # genera respuestas basadas en RAG

# Flujo: START -> agent -> execute -> (generate o agent)
builder.add_edge(START, "agent")

builder.add_conditional_edges(
    "agent",
    tools_condition,
    {
        "tools": "execute",
        END: END,
    },
)

builder.add_conditional_edges(
    "execute",
    validar_rag,
    {
        "generate": "generate",
        "agent": "agent"
    }
)

# El nodo generate regresa al agent para continuar la conversación
builder.add_edge("generate", "agent")

graph_name = os.getenv("GRAPH_NAME", "chatbot_graph_v1")

graph = builder.compile(
    checkpointer=within_thread_memory,
    name=graph_name,
)
graph

In [ ]:
# Función para procesar mensajes (similar a la anterior)
@traceable(name="process_message", metadata={"component": "adaptive_rag_chatbot"})
def process_message_v2(message: str, user_id: str) -> str:
    config = {"configurable": {"thread_id": user_id, "user_id": user_id, "checkpoint_ns": graph_name}}
    input_messages = [HumanMessage(content=message)]

    # Inicializar el estado
    initial_state = {
        "messages": input_messages,
    }

    result = graph.invoke(initial_state, config)
    ai_message = result["messages"][-1]

    return ai_message.content


# Pruebas

## Prueba 1

In [ ]:
respuesta = process_message_v2("que programas hay?", "ernesto")

In [ ]:
print(respuesta)

## Prueba 2

In [ ]:
respuesta = process_message_v2("cual es el costo del programa Data Engineer?", "ernesto")

In [ ]:
print(respuesta)

## Prueba 3

In [ ]:
respuesta = process_message_v2("Que programas hay y cual es el costo de cada uno?", "ernesto")

In [ ]:
print(respuesta)